In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [1]:
!pip uninstall -y tensorflow tensorflow-intel tf-nightly
!pip install tensorflow==2.16.1

Found existing installation: tensorflow 2.16.1
Uninstalling tensorflow-2.16.1:
  Successfully uninstalled tensorflow-2.16.1
  Using cached tensorflow-2.16.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
Using cached tensorflow-2.16.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tensorflow==2.19.0, but you have tensorflow 2.16.1 which is incompatible.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.16.1 which is incompatible.
tensorflow-text 2.19.0 requires tensorflow<2.20,>=2.19.0, but you have tensorflow 2.16.1 which is incompatible.


In [2]:
import tensorflow as tf
print(tf.__version__)  # debería mostrar 2.16.1

2.16.1


In [3]:
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.1


In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-09-15 10:43:31--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.05s   

2025-09-15 10:43:32 (7.14 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2025-09-15 10:43:32--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [5]:
# Reading the data with pandas
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["label", "message"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["label", "message"])

In [6]:
train_df.head()

,label,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [7]:
test_df.head()

,label,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [8]:
train_df.shape

(4179, 2)

In [9]:
test_df.shape

(1392, 2)

In [10]:
# Seeing the number of label in train_df
train_df.label.value_counts()

,count
label,
ham,3619
spam,560


In [11]:
# Seeing the number of label in test_df
test_df.label.value_counts()

,count
label,
ham,1205
spam,187


In [12]:
# Preprocess labels (ham=0, spam=1)
train_labels = train_df['label'].map({'ham': 0, 'spam': 1}).values
test_labels = test_df['label'].map({'ham': 0, 'spam': 1}).values

In [13]:
# Preprocess text data. Hyperparameters
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 100

In [14]:
# Create TextVectorization layer\n",
vectorizer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH
    )

vectorizer.adapt(train_df['message'].values)

In [15]:
# Convert text to sequences
train_sequences = vectorizer(train_df['message'].values)
test_sequences = vectorizer(test_df['message'].values)

In [16]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_sequences, train_labels)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((test_sequences, test_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

In [17]:
# Build the model
def build_model():

  model = keras.Sequential([
      tf.keras.layers.Embedding(VOCAB_SIZE, 64, input_length=MAX_SEQUENCE_LENGTH),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
      tf.keras.layers.GlobalMaxPool1D(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])

  model.compile(
      optimizer='adam',
      loss='binary_crossentropy',
      metrics=['accuracy']
      )

  return model



In [18]:
model = build_model()
model.build(input_shape=(None, MAX_SEQUENCE_LENGTH))  # batch_size = None
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 64)        │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 128)       │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 128)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 714,369 (2.73 MB)

 Trainable params: 714,369 (2.73 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=10,
    validation_data=test_dataset,
    verbose=1
    )

Epoch 1/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 20s 118ms/step - accuracy: 0.8480 - loss: 0.4403 - val_accuracy: 0.9677 - val_loss: 0.1362
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 109ms/step - accuracy: 0.9674 - loss: 0.1259 - val_accuracy: 0.9734 - val_loss: 0.1092
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 108ms/step - accuracy: 0.9870 - loss: 0.0633 - val_accuracy: 0.9792 - val_loss: 0.0768
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 16s 120ms/step - accuracy: 0.9946 - loss: 0.0296 - val_accuracy: 0.9871 - val_loss: 0.0629
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 19s 109ms/step - accuracy: 0.9975 - loss: 0.0093 - val_accuracy: 0.9864 - val_loss: 0.0855
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 16s 120ms/step - accuracy: 0.9993 - loss: 0.0064 - val_accuracy: 0.9849 - val_loss: 0.0966
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 25s 158ms/step - accuracy: 0.9869 - loss: 0.0421 - val_accuracy: 0.9842 - val_loss: 0.0626
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 14s 109ms/step - accuracy: 0.9976 - loss: 0

In [20]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):

  # Convert text to sequence
  sequence = vectorizer([pred_text])

  # Get probability of spam (1)
  prob = model.predict(sequence, verbose=0)[0][0]

  # Determine label based on probability threshold
  label = 'spam' if prob >= 0.5 else 'ham'

  return [prob, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[8.0777536e-05, 'ham']


In [21]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
